In [8]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import torch
import cv2
import numpy as np
import torch
from rlepose.models import builder
from rlepose.utils.config import update_config
from rlepose.utils.valid_utils_lxd import calculate_error_distance_avg, calculate_RMSE, calculate_PCK, paint


cfg = update_config("../configs/256x192_res50_regress-flow_freihand.yaml")
device = torch.device('cuda')
# m = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)  # 根据cfg的配置信息构建模型

# m.load_state_dict(torch.load("/home/louxd/ldlib/res-loglikelihood-regression-master/weights/model_0918_233.pth", map_location='cpu'), 
#                   strict=True)  # 加载权重

model_path = './hand_landmarker.task'

base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)


with torch.no_grad():
    gt_val_dataset = builder.build_dataset(cfg.DATASET.VAL, preset_cfg=cfg.DATA_PRESET, train=False, heatmap2coord=cfg.TEST.HEATMAP2COORD)
    gt_val_sampler = torch.utils.data.distributed.DistributedSampler(
        gt_val_dataset, num_replicas=1, rank=0)

    gt_val_loader = torch.utils.data.DataLoader(
        gt_val_dataset, batch_size=16, shuffle=False, num_workers=20, drop_last=False, sampler=gt_val_sampler)
    
    
    rmse_list = []
    pck_pix = []
    pck_norm = []
    err_pix = []

    for index, (inps, labels) in enumerate(gt_val_loader):
        inps = inps.cuda(device)
        for k, _ in labels.items():
            if k == 'type':
                continue
            
            labels[k] = labels[k].cuda(device)

        # output = m(inps, labels)
        for i in range(len(inps)):
            imgi = inps[i].cpu().numpy()
            imgi = np.transpose(imgi, (1, 2, 0))
            imgi = (imgi + np.array([0.480, 0.457, 0.406], dtype=np.float32)) * np.array([255., 255., 255.], dtype=np.float32)
            imgi = cv2.cvtColor(imgi, cv2.COLOR_BGR2RGB) 
            imgi = imgi.astype(np.uint8)
            img_h, img_w, _ = imgi.shape
            # kpts_pre_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_pre[i]])
            # kpts_gt_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_gt[i]])
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=imgi)
            detection_result = detector.detect(mp_image)
            hand_lm = detection_result.hand_landmarks[0]
            pts_i = []

            for lm in hand_lm:
                x = lm.x * img_w
                y = lm.y * img_h
                pts_i.append([x, y])

            imagei_pre = imgi.copy()
            # imagei_gt = imgi.copy()
            imagei_pre = paint(imagei_pre, pts_i)
            # imagei_gt = paint(imagei_gt, kpts_gt_i)
            cv2.imwrite(f'./exp/output/{index}_{i}_pre.jpg', imagei_pre)
            cv2.imwrite(f'./exp/output/{index}_{i}_gt.jpg', imagei_gt)

            # cal mAP and rmse
            # rmsei = compute_RMSE(kpts_pre_i, kpts_gt_i)
            # mse_i = mean_squared_error(kpts_pre_i, kpts_gt_i)
            # rmse_i = np.sqrt(mse_i)
            # _, rmse_i = calculate_RMSE(kpts_pre_i, kpts_gt_i)
            # # oksi = calculate_oks_pt2(kpts_pre_i, kpts_gt_i)
            # pck_pix_i, pck_norm_i = calculate_PCK(kpts_pre_i, kpts_gt_i)
            # err_pix_i = calculate_error_distance_avg(kpts_pre_i, kpts_gt_i)
            # rmse_list.append(rmse_i)
            # pck_pix.append(pck_pix_i)
            # pck_norm.append(pck_norm_i)
            # err_pix.append(err_pix_i)









    #     if isinstance(inps, list):
    #         batch_size = inps[0].size(0)
    #     else:
    #         batch_size = inps.size(0)

    #     kpts_gt = labels['target_uv'].cpu().numpy().reshape(-1, 21, 2) # size * 42
    #     kpts_pre = output.pred_jts.cpu().numpy().reshape(-1, 21, 2)

    #     # print & draw
    #     for i in range(len(inps)):
    #         #print(f'batch:{batch_idx}_{i}')
    #         imgi = inps[i].cpu().numpy()
    #         imgi = np.transpose(imgi, (1, 2, 0))
    #         imgi = (imgi + np.array([0.480, 0.457, 0.406], dtype=np.float32)) * np.array([255., 255., 255.], dtype=np.float32)
    #         imgi = cv2.cvtColor(imgi, cv2.COLOR_BGR2RGB) 
    #         img_h, img_w, _ = imgi.shape
    #         kpts_pre_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_pre[i]])
    #         kpts_gt_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_gt[i]])


    #         # cal mAP and rmse
    #         # rmsei = compute_RMSE(kpts_pre_i, kpts_gt_i)
    #         # mse_i = mean_squared_error(kpts_pre_i, kpts_gt_i)
    #         # rmse_i = np.sqrt(mse_i)
    #         _, rmse_i = calculate_RMSE(kpts_pre_i, kpts_gt_i)
    #         # oksi = calculate_oks_pt2(kpts_pre_i, kpts_gt_i)
    #         pck_pix_i, pck_norm_i = calculate_PCK(kpts_pre_i, kpts_gt_i)
    #         err_pix_i = calculate_error_distance_avg(kpts_pre_i, kpts_gt_i)
    #         rmse_list.append(rmse_i)
    #         pck_pix.append(pck_pix_i)
    #         pck_norm.append(pck_norm_i)
    #         err_pix.append(err_pix_i)

    #         # imagei_pre = imgi.copy()
    #         # imagei_gt = imgi.copy()
    #         # imagei_pre = paint(imagei_pre, kpts_pre_i)
    #         # imagei_gt = paint(imagei_gt, kpts_gt_i)
    #         # cv2.imwrite(f'./exp/output/{index}_{i}_pre.jpg', imagei_pre)
    #         # cv2.imwrite(f'./exp/output/{index}_{i}_gt.jpg', imagei_gt)


    # rmse = sum(rmse_list) / len(rmse_list)
    # pck_pix = np.mean(pck_pix, axis=0)
    # pck_norm = np.mean(pck_norm, axis=0)
    # err_pix = np.mean(err_pix)
    # # mAP_info_str = calculate_mAP(oks_list)

    # # ap = mAP_info_str['mAP']




anno_xyz len： 3960
anno_K len： 3960
img list len： 3960
AUG OFF.


RuntimeError: No active exception to reraise

In [31]:
a = detection_result.hand_landmarks[0]
rescaled_result = []
for lm in a:
    x = lm.x * img_w
    y = lm.y * img_h
    rescaled_result.append([x, y])


NormalizedLandmark(x=0.8822426199913025, y=0.49387818574905396, z=8.636168900011398e-07, visibility=0.0, presence=0.0)


TypeError: 'NormalizedLandmark' object is not iterable

In [20]:
detection_result.hand_landmarks[0][0]

NormalizedLandmark(x=0.8822426199913025, y=0.49387818574905396, z=8.636168900011398e-07, visibility=0.0, presence=0.0)